# Imports

In [ ]:
# Python standard library
import sys
import csv

# Scientific computing
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.ndimage

# Program specific
sys.path.append('/home/prestonh/Desktop/Research/pore_stats/pore_stats/rp/')
sys.path.append('/home/prestonh/Desktop/Research/pore_stats/pore_stats/oi/')
import resistive_pulse as rp
import rp_file
import optical_imaging as oi
import oi_file


# Jupyter
from IPython.display import HTML

# Load data

In [ ]:
date = '7-17-2017/'
particle = '293-T/'
channel = '15-30-15_1/'
file_index = '0'

base_path = '/home/prestonh/Desktop/Research/cancer_cells/data/'

oi_vid_file_path = base_path + date + particle + channel + 'oi/bin/test_camera_' + file_index
oi_events_file_path = base_path + date + particle + channel + 'oi/events/test_camera_' + file_index + '_events_sync.json'

In [ ]:
# Load video
oi_vid = oi_file.Video(oi_vid_file_path, 512, 288, 100000)

# Load events
oi_events = oi_file.open_event_file_json(oi_events_file_path)

# Load template and create stage

In [ ]:
template_index = 0

template_frame = oi_vid.get_frame(template_index)

plt.imshow(template_frame, cmap = 'gray', origin = 'lower')
plt.show()

In [ ]:
reload(oi)
template_index = 0
template_frame = oi_vid.get_frame(template_index)
c0 = [118,88]
c1 = [119,61]
c2 = [360,60]
c3 = [360,86]
oi_stage = oi.Stage(template_frame, c0, c1, c2, c3)
oi_stage.plot_stage()

# Define plotting functions

In [ ]:
def plot_frame(frame, show = True):
    plt.imshow(frame, cmap = 'gray', origin = 'lower', interpolation = 'none')
    
    if show:
        plt.show()
    
def plot_processed_over_raw(processed_frame, frame, show = True):
    green_processed_frame = np.zeros((processed_frame.shape[0], processed_frame.shape[1], 3))
    green_processed_frame[:,:,1] = processed_frame

    plt.imshow(frame, cmap = 'gray', origin = 'lower', alpha = 1, interpolation = 'none')
    plt.imshow(green_processed_frame, cmap = 'gray', origin = 'lower', alpha = 0.15, interpolation = 'none')

    if show:
        plt.show()
    
def plot_frame_hist(frame, show = True):
    plt.hist(frame.flatten(), facecolor = 'k', bins = 100)
    
    if show:
        plt.show()
    
def plot_all(processed_frame, frame):
    fig, axes = plt.subplots(1,3,figsize = (12,3))
    
    # Plot 1
    plt.sca(axes[0])
    plot_frame(processed_frame, show = False)
    
    
    # Plot 2
    plt.sca(axes[1])
    plot_processed_over_raw(processed_frame, frame, show = False)
    
    # Plot 3
    plt.sca(axes[2])
    plot_frame_hist(processed_frame, show = False)
    
    plt.show()

# Filter out bad events

In [ ]:
filtering_steps = []

### Manual filtering

In [ ]:
remove_indices = []

filtering_steps.append(['manual removal (num events manually removed)', len(remove_indices)])

oi_events = [oi_events[i] for i in range(len(oi_events)) if i not in remove_indices]



### y-based filtering

In [ ]:
# Central cavity y-based filtering

yc_middles = []

for oi_event in oi_events:
    
    # Calculate y at center of channel
    xs = oi_event.get_px()
    ys = oi_event.get_py()
    tfs = oi_event.get_tf()
    tfs = np.array(tfs)
    tfs = tfs - tfs[0]
    
    xcs, ycs = oi_stage.get_channel_coordinates(xs, ys)
    xcs = oi_stage.pixels_to_meters(xcs)
    ycs = oi_stage.pixels_to_meters(ycs)
    
    tfs_interp = scipy.interpolate.interp1d(xcs, tfs)
    
    tf_middle = int(tfs_interp(oi_stage._length_microns/2.))
    
    
    yc_middles.append(ycs[tf_middle])
    
    
    
    
# Apply offset to y so centered at 0
offset = -(np.max(yc_middles) - np.abs(np.min(yc_middles)))/2.
yc_middles = np.array(yc_middles)
yc_middles = yc_middles + offset


# Plot hist of all ys
plt.hist(yc_middles)
plt.show()

In [ ]:
# Filter y


yc_threshold = 5
filtering_steps.append(['yc cavity threshold', yc_threshold])

keep_indices = np.where(np.abs(yc_middles) <= yc_threshold)[0]

print 'length before', len(oi_events), 'length after', len(oi_events) - len(keep_indices)



oi_events = [oi_events[i] for i in keep_indices]

### Size-based filtering

In [ ]:
preprocessing_steps = []
def preprocess_frame(raw_frame, template_frame, detection, debug = False):

    # Refresh list
    del preprocessing_steps[:]

    
    
    
    #################################
    # Crop raw frame
    #################################


    x0 = int(detection._px - 30)
    x1 = int(detection._px + 30)
    y0 = int(detection._py - 30)
    y1 = int(detection._py + 30)
    frame = raw_frame[y0:y1, x0:x1]

    
    
    
    
    #################################
    # Copy
    #################################




    preprocessing_steps.append(['copy'])

    processed_frame = 1.*frame

    if debug:
        print 'copy'
        plot_all(processed_frame, frame)


    #################################
    # Negative
    #################################



    preprocessing_steps.append(['negative'])

    processed_frame = np.abs(frame - template_frame[y0:y1, x0:x1])


    if debug:
        print 'negative'
        plot_all(processed_frame, frame)




    #################################
    # Blur
    #################################


    blur_kernel = (3,3)
    preprocessing_steps.append(['gaussian blur', blur_kernel])

    processed_frame = cv2.GaussianBlur(processed_frame, blur_kernel, 0)



    if debug:
        print 'gaussian blur'
        plot_all(processed_frame, frame)





    #################################
    # Rescale
    #################################

    preprocessing_steps.append(['rescale'])

    processed_frame = (processed_frame - np.min(processed_frame))/(np.max(processed_frame) - np.min(processed_frame))



    if debug:
        print 'rescale'
        plot_all(processed_frame, frame)






    #################################
    # Threshold
    #################################

    threshold = .15
    preprocessing_steps.append(['pixel intensity threshold', threshold])


    processed_frame[processed_frame > threshold] = 1
    processed_frame[processed_frame <= threshold] = 0




    if debug:
        print 'threshold'
        plot_all(processed_frame, frame)




    #################################
    # Erode subtraction
    #################################
    iterations = 1
    preprocessing_steps.append(['erode subtraction, iterations = ', iterations])


    processed_frame = processed_frame - scipy.ndimage.morphology.binary_erosion(processed_frame, iterations = 1)




    if debug:
        print 'erode subtraction'
        plot_all(processed_frame, frame)




    #################################
    # Fill holes
    #################################
    preprocessing_steps.append(['fill holes'])

    processed_frame = scipy.ndimage.binary_fill_holes(processed_frame)




    if debug:
        print 'fill holes'
        plot_all(processed_frame, frame)

    #################################    
    # Get largest cluster
    #################################

    preprocessing_steps.append(['clustering'])


    clusters = oi.find_clusters_percentage_based(processed_frame, np.zeros((processed_frame.shape[0], processed_frame.shape[1])), diag = True)
    largest_cluster = sorted(clusters, key = lambda x: len(x))[-1]
    processed_frame = np.zeros(processed_frame.shape, dtype = np.uint8)
    for pixel in largest_cluster:
        processed_frame[pixel[0], pixel[1]] = 1





    if debug:
        print 'cluster'
        plot_all(processed_frame, frame)


    #################################
    # Morphological closing
    #################################

    morph_kernel_size = (10,10)
    preprocessing_steps.append(['morphological closing, kernel = ', morph_kernel_size])

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, morph_kernel_size)
    processed_frame = cv2.morphologyEx(np.array(processed_frame, dtype = np.uint8), cv2.MORPH_CLOSE, kernel)


    if debug:
        print 'morphological closing'
        plot_all(processed_frame, frame)




    #################################
    # Erode subtraction
    #################################

    iterations = 3
    preprocessing_steps.append(['erosion subtraction, iterations = ', iterations])



    processed_frame = processed_frame - scipy.ndimage.morphology.binary_erosion(processed_frame, iterations = iterations)


    if debug:
        print 'erode subtraction'
        plot_all(processed_frame, frame)



    #################################
    # Fit ellipse
    #################################



    cell_pixels = np.where(processed_frame == 1)

    ellipse = oi.fit_ellipse(cell_pixels[0], cell_pixels[1])
    
    
    if debug:
        ellipse_center = oi.get_ellipse_center(ellipse)
        ellipse_angle = oi.get_ellipse_angle_of_rotation(ellipse)
        ellipse_axes = oi.get_ellipse_axes_lengths(ellipse)





        ellipse_points = np.empty((100,2))
        for i in range(100):
            angle = i*2*np.pi/99.
            x = ellipse_axes[0]*np.cos(angle)
            y = ellipse_axes[1]*np.sin(angle)
            ellipse_points[i,1] = ellipse_center[0] + np.cos(ellipse_angle)*x - np.sin(ellipse_angle)*y
            ellipse_points[i,0] = ellipse_center[1] + np.sin(ellipse_angle)*x + np.cos(ellipse_angle)*y


        green_processed_frame = np.zeros((processed_frame.shape[0], processed_frame.shape[1], 3))
        green_processed_frame[:,:,1] = processed_frame



        plt.imshow(frame, cmap = 'gray', origin = 'lower', interpolation = 'none')
        plt.imshow(green_processed_frame, alpha = .15, origin = 'lower', interpolation = 'none')
        plt.plot(ellipse_points[:,0], ellipse_points[:,1], lw = 0.5, c = 'yellow')

        plt.xlim(0, frame.shape[1])
        plt.ylim(0, frame.shape[0])

        plt.show()
        
    return ellipse


In [ ]:
effective_radii = []

for oi_event in oi_events:
    
    
    
    xs = oi_event.get_px()
    ys = oi_event.get_py()
    
    xcs, ycs = oi_stage.get_channel_coordinates(xs, ys)
    xcs = oi_stage.pixels_to_meters(xcs)
    
    index = np.where((xcs > -30) & (xcs < -10))[0][0]
    
    detection = oi_event._detections[index]
    
    frame = oi_vid.get_frame(detection._tf)
        
    ellipse = preprocess_frame(frame, template_frame, detection, debug = False)
    
    
    
    ellipse_axes = oi.get_ellipse_axes_lengths(ellipse)
    
    ellipse_axes[0] = oi_stage.pixels_to_meters(ellipse_axes[0])
    ellipse_axes[1] = oi_stage.pixels_to_meters(ellipse_axes[1])
    
    effective_radii.append(np.sqrt(ellipse_axes[0]*ellipse_axes[1]))
    
plt.hist(effective_radii, bins = 20)
plt.show()

In [ ]:
radius_threshold = 11

filtering_steps.append(['effective radius threshold', radius_threshold])

filter_indices = np.where(np.array(effective_radii) > radius_threshold)[0]

print filter_indices

In [ ]:
oi_index = 2
reload(oi_file)
plt.close()
HTML(oi_file.make_animation(oi_vid, oi_events[oi_index]._detections[0]._tf, oi_events[oi_index]._detections[-1]._tf).to_html5_video())

In [ ]:
print 'length before', len(oi_events), 'length after', len(oi_events) - len(filter_indices)
oi_events = [oi_events[i] for i in range(len(oi_events)) if i not in filter_indices]

# Find a good event to test fitting parameters

In [ ]:
oi_index = 16
reload(oi_file)
plt.close()
HTML(oi_file.make_animation(oi_vid, oi_events[oi_index]._detections[0]._tf, oi_events[oi_index]._detections[-1]._tf).to_html5_video())

# Negative thresholding

In [ ]:
preprocessing_steps = []
def preprocess_frame(raw_frame, template_frame, detection, debug = False):

    # Refresh list
    del preprocessing_steps[:]

    
    
    
    #################################
    # Crop raw frame
    #################################


    x0 = int(detection._px - 30)
    x1 = int(detection._px + 30)
    y0 = int(detection._py - 30)
    y1 = int(detection._py + 30)
    frame = raw_frame[y0:y1, x0:x1]

    
    
    
    
    #################################
    # Copy
    #################################




    preprocessing_steps.append(['copy'])

    processed_frame = 1.*frame

    if debug:
        print 'copy'
        plot_all(processed_frame, frame)


    #################################
    # Negative
    #################################



    preprocessing_steps.append(['negative'])

    processed_frame = np.abs(frame - template_frame[y0:y1, x0:x1])


    if debug:
        print 'negative'
        plot_all(processed_frame, frame)




    #################################
    # Blur
    #################################


    blur_kernel = (3,3)
    preprocessing_steps.append(['gaussian blur', blur_kernel])

    processed_frame = cv2.GaussianBlur(processed_frame, blur_kernel, 0)



    if debug:
        print 'gaussian blur'
        plot_all(processed_frame, frame)





    #################################
    # Rescale
    #################################

    preprocessing_steps.append(['rescale'])

    processed_frame = (processed_frame - np.min(processed_frame))/(np.max(processed_frame) - np.min(processed_frame))



    if debug:
        print 'rescale'
        plot_all(processed_frame, frame)






    #################################
    # Threshold
    #################################

    threshold = .15
    preprocessing_steps.append(['pixel intensity threshold', threshold])


    processed_frame[processed_frame > threshold] = 1
    processed_frame[processed_frame <= threshold] = 0




    if debug:
        print 'threshold'
        plot_all(processed_frame, frame)




    #################################
    # Erode subtraction
    #################################
    iterations = 1
    preprocessing_steps.append(['erode subtraction, iterations = ', iterations])


    processed_frame = processed_frame - scipy.ndimage.morphology.binary_erosion(processed_frame, iterations = 1)




    if debug:
        print 'erode subtraction'
        plot_all(processed_frame, frame)




    #################################
    # Fill holes
    #################################
    preprocessing_steps.append(['fill holes'])

    processed_frame = scipy.ndimage.binary_fill_holes(processed_frame)




    if debug:
        print 'fill holes'
        plot_all(processed_frame, frame)

    #################################    
    # Get largest cluster
    #################################

    preprocessing_steps.append(['clustering'])


    clusters = oi.find_clusters_percentage_based(processed_frame, np.zeros((processed_frame.shape[0], processed_frame.shape[1])), diag = True)
    largest_cluster = sorted(clusters, key = lambda x: len(x))[-1]
    processed_frame = np.zeros(processed_frame.shape, dtype = np.uint8)
    for pixel in largest_cluster:
        processed_frame[pixel[0], pixel[1]] = 1





    if debug:
        print 'cluster'
        plot_all(processed_frame, frame)


    #################################
    # Morphological closing
    #################################

    morph_kernel_size = (10,10)
    preprocessing_steps.append(['morphological closing, kernel = ', morph_kernel_size])

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, morph_kernel_size)
    processed_frame = cv2.morphologyEx(np.array(processed_frame, dtype = np.uint8), cv2.MORPH_CLOSE, kernel)


    if debug:
        print 'morphological closing'
        plot_all(processed_frame, frame)




    #################################
    # Erode subtraction
    #################################

    iterations = 3
    preprocessing_steps.append(['erosion subtraction, iterations = ', iterations])



    processed_frame = processed_frame - scipy.ndimage.morphology.binary_erosion(processed_frame, iterations = iterations)


    if debug:
        print 'erode subtraction'
        plot_all(processed_frame, frame)



    #################################
    # Fit ellipse
    #################################



    cell_pixels = np.where(processed_frame == 1)

    ellipse = oi.fit_ellipse(cell_pixels[0], cell_pixels[1])
    
    
    if debug:
        ellipse_center = oi.get_ellipse_center(ellipse)
        ellipse_angle = oi.get_ellipse_angle_of_rotation(ellipse)
        ellipse_axes = oi.get_ellipse_axes_lengths(ellipse)





        ellipse_points = np.empty((100,2))
        for i in range(100):
            angle = i*2*np.pi/99.
            x = ellipse_axes[0]*np.cos(angle)
            y = ellipse_axes[1]*np.sin(angle)
            ellipse_points[i,1] = ellipse_center[0] + np.cos(ellipse_angle)*x - np.sin(ellipse_angle)*y
            ellipse_points[i,0] = ellipse_center[1] + np.sin(ellipse_angle)*x + np.cos(ellipse_angle)*y


        green_processed_frame = np.zeros((processed_frame.shape[0], processed_frame.shape[1], 3))
        green_processed_frame[:,:,1] = processed_frame



        plt.imshow(frame, cmap = 'gray', origin = 'lower', interpolation = 'none')
        plt.imshow(green_processed_frame, alpha = .15, origin = 'lower', interpolation = 'none')
        plt.plot(ellipse_points[:,0], ellipse_points[:,1], lw = 0.5, c = 'yellow')

        plt.xlim(0, frame.shape[1])
        plt.ylim(0, frame.shape[0])

        plt.show()
        
    return ellipse


In [ ]:
oi_event = oi_events[oi_index]
detection = oi_event._detections[45]
raw_frame = oi_vid.get_frame(detection._tf)

plt.imshow(raw_frame, cmap = 'gray', origin = 'lower')
plt.show()

x0 = int(detection._px - 30)
x1 = int(detection._px + 30)
y0 = int(detection._py - 30)
y1 = int(detection._py + 30)
plt.imshow(raw_frame[y0:y1, x0:x1], cmap = 'gray', origin = 'lower', interpolation = 'none')
plt.show()

In [ ]:
preprocess_frame(raw_frame, template_frame, detection, debug = True)

# Preprocess, fit ellipse to all detections, calculate aspect ratios at every frame

In [ ]:
oi_event = oi_events[7]

oi_event.get_channel_enter_exit_tf(oi_stage)


xcs = []
aspects = []


for i in range(oi_event._channel_enter_tf-5, oi_event._channel_exit_tf-1):
    detection = oi_event._detections[i - oi_event._detections[0]._tf]
    raw_frame = oi_vid.get_frame(detection._tf)
    
    ellipse = preprocess_frame(raw_frame, template_frame, detection, debug = False)
    
    ellipse_axes = oi.get_ellipse_axes_lengths(ellipse)
    
    x = detection._px
    y = detection._py
    
    xc, yc = oi_stage.get_channel_coordinates(x, y)
    xc = oi_stage.pixels_to_meters(xc)
    xcs.append(xc/oi_stage._length_microns)
    aspects.append(ellipse_axes[0]/ellipse_axes[1])
    
plt.plot(xcs, aspects)
plt.plot([np.array(xcs).min(),np.array(xcs).max()], [1,1], lw = 2, ls = '--')
plt.show()

### Find maximum and minimum aspect ratios

In [ ]:
def find_max_min_aspects(aspects, xcs, debug = False):
    # Smooth the aspect ratio signal
    sigma = 2
    aspects_smoothed = scipy.ndimage.filters.gaussian_filter(aspects, sigma = sigma)

    # Find minima and maxima in the aspect ratios
    maxima, minima = rp.get_maxima_minima(aspects, num_maxima = 1, num_minima = 1, sigma = sigma, return_by = ['high', 'low'])

    maxima = maxima[0]
    minima = minima[0]


    # Average in a window around the local minima and maxima
    window_size = 3
    max_aspect = np.mean(aspects[maxima-window_size/2:maxima+window_size/2+1])
    min_aspect = np.mean(aspects[minima-window_size/2:minima+window_size/2+1])






    if debug:
        # Create plot

        # Plot aspects, smoothed aspects
        plt.plot(xcs, aspects, lw = 3, c = 'k')
        plt.plot(xcs, aspects_smoothed, c = 'gray', ls = '--', lw = 5)


        # Plot the minima/maxima as detected in the smoothed signal
        plt.scatter(xcs[maxima], aspects_smoothed[maxima], s = 150, marker = 'x', lw = 5, color = 'red')
        plt.scatter(xcs[minima], aspects_smoothed[minima], s = 150, marker = 'x', lw = 5, color = 'red')

        # Plot the averaged minima/maxima
        plt.plot([xcs[0], xcs[-1]], [max_aspect, max_aspect], ls = '--', lw = 3, c = 'red')
        plt.plot([xcs[0], xcs[-1]], [min_aspect, min_aspect], ls = '--', lw = 3, c = 'red')
        plt.plot([xcs[0], xcs[-1]], [1,1], ls = '--', lw = 3, c = 'k')

        plt.xlim(np.min(xcs), np.max(xcs))

        # Cosmetics
        plt.xlabel(r'axial position $x_{c}/L_{c}$', size = 12)
        plt.ylabel(r'aspect ratio (>1 = oblate)', size = 12)
        plt.grid()






        plt.show()
        
    return max_aspect, min_aspect

In [ ]:
find_max_min_aspects(aspects, xcs, debug = True)

### Calculate minimum and maximum aspect for all events

In [ ]:
max_aspects = []
min_aspects = []

aspectss = []

for i, oi_event in enumerate(oi_events):

    print 'event #', i, '/', len(oi_events)
    
    try:
        oi_event.get_channel_enter_exit_tf(oi_stage)


        xcs = []
        aspects = []


        for j in range(oi_event._channel_enter_tf-5, oi_event._channel_exit_tf+5):
            detection = oi_event._detections[j - oi_event._detections[0]._tf]
            raw_frame = oi_vid.get_frame(detection._tf)

            ellipse = preprocess_frame(raw_frame, template_frame, detection, debug = False)

            ellipse_axes = oi.get_ellipse_axes_lengths(ellipse)

            x = detection._px
            y = detection._py

            xc, yc = oi_stage.get_channel_coordinates(x, y)
            xc = oi_stage.pixels_to_meters(xc)
            xcs.append(xc/oi_stage._length_microns)
            aspects.append(ellipse_axes[0]/ellipse_axes[1])
        
            
            

        max_aspect, min_aspect = find_max_min_aspects(aspects, xcs, debug = True)

        max_aspects.append(max_aspect)
        min_aspects.append(min_aspect)
        aspectss.append(aspects)
        
    except: 
        print 'failed on event', i

In [ ]:
plt.scatter(min_aspects, max_aspects)

plt.plot([0,10],[1,1], c = 'k', ls = ':')
plt.plot([1,1],[0,10], c = 'k', ls = ':')

plt.xlabel('min aspect')
plt.ylabel('max aspect')

plt.xlim(0,2)
plt.ylim(0,2)

plt.show()

In [ ]:
plt.hist(min_aspects)
plt.show()

plt.hist(max_aspects)
plt.show()

# Save data and parameters

In [ ]:
output_file_path = '../data/' + date + particle + channel + 'aspects_' + file_index
print output_file_path


with open(output_file_path, 'w') as file_handle:
    file_writer = csv.writer(file_handle, delimiter = '\t')
    
    
    # Write filtering steps
    for filtering_step in filtering_steps:
        file_writer.writerow(filtering_step)
        
    # Write preprocessing steps
    for preprocessing_step in preprocessing_steps:
        file_writer.writerow(preprocessing_step)
    
    # Write data
    for i in range(len(aspectss)):
        file_writer.writerow(aspectss)